# **DATA301 Data Project**

The data project revolves around the reddit submission [dataset](https://snap.stanford.edu/data/web-Reddit.html). The purpose of this project is to find....

In [ ]:
#import necessary items
import dask
from dask import bag as db
from dask import dataframe as df
from sklearn.feature_extraction.text import TfidfVectorizer

#downloading stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
custom_stopwords = {'just', 'im', 'the'}
stop_words1 = stop_words.union(custom_stopwords)
print(stop_words1)


{'against', 'been', "weren't", 'few', "didn't", 't', "won't", 'more', 'so', 'own', 'yourselves', "you'll", "she'd", 'i', 'mightn', 'not', 'too', "needn't", "they're", 'them', 'after', "i'd", 'same', 'about', 'd', "they'd", 'itself', 'other', "we'd", 'there', 'through', 'had', "couldn't", "don't", "i'll", "she'll", 'nor', 're', 'doing', 'can', 'again', 'down', 'from', 'isn', "mustn't", "you've", 'him', 'or', 'where', 'were', 'each', 'o', 'by', 'aren', 'out', 'your', "shouldn't", "i've", "i'm", 'should', 'ourselves', 'why', 'what', 'me', 'didn', 'that', 'theirs', 'themselves', 'yourself', 'myself', "aren't", 'to', 'between', 'y', "it's", 'be', 'which', 'those', 'while', "hadn't", "we're", 'couldn', 'just', 'some', 'under', 'ain', 'has', "they've", 'on', 'whom', "that'll", 'with', 'you', "he'll", 'her', 'any', 'they', 'who', 'how', 'herself', 'all', "doesn't", 'further', 'and', 'an', 'our', 'his', 'do', 'is', 'having', 'of', 'over', 'shouldn', "wasn't", 'was', "hasn't", 'wouldn', 'their',

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Setting up environment**
Downloading the dataset

In [ ]:

import gzip
import os

if os.path.exists('redditSubmissions.csv.gz'):
  print('File already exists')
else:
  !wget https://snap.stanford.edu/data/redditSubmissions.csv.gz
  with gzip.open('redditSubmissions.csv.gz', 'rb') as data_in:
    with open('submissions.csv', 'wb') as data_out:
      data_out.write(data_in.read())
  print('File downloaded')



File already exists


In [ ]:
#reading the dataset
#lines that don't match the data structure are skipped
dataset = df.read_csv('submissions.csv', on_bad_lines='warn',
                      dtype={'localtime': 'float64',
                             'number_of_comments': 'float64',
                             'number_of_downvotes': 'float64',
                             'number_of_upvotes': 'float64',
                             'score': 'float64',
                             'total_votes': 'float64',
                             'unixtime': 'float64'})
print(dataset)
print(dataset.head())


Dask DataFrame Structure:
              #image_id unixtime rawtime   title total_votes reddit_id number_of_upvotes subreddit number_of_downvotes localtime    score number_of_comments username
npartitions=1                                                                                                                                                        
                  int64  float64  string  string     float64    string           float64    string             float64   float64  float64            float64   string
                    ...      ...     ...     ...         ...       ...               ...       ...                 ...       ...      ...                ...      ...
Dask Name: to_string_dtype, 2 expressions
Expr=ArrowStringConversion(frame=FromMapProjectable(34ca8c0))


/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


   #image_id      unixtime                           rawtime  \
0          0  1.333172e+09  2012-03-31T12:40:39.590113-07:00   
1          0  1.333178e+09  2012-03-31T14:16:01.093638-07:00   
2          0  1.333200e+09  2012-03-31T20:18:33.192906-07:00   
3          0  1.333252e+09         2012-04-01T10:52:10-07:00   
4          0  1.333273e+09  2012-04-01T16:35:54.393381-07:00   

                             title  total_votes reddit_id  number_of_upvotes  \
0           And here's a downvote.      63470.0     rmqjs            32657.0   
1                      Expectation         35.0     rmun4               29.0   
2                         Downvote         41.0     rna86               32.0   
3  Every time I downvote something         10.0     ro7e4                6.0   
4   Downvote &quot;Dies Irae&quot;         65.0     rooof               57.0   

  subreddit  number_of_downvotes     localtime   score  number_of_comments  \
0     funny              30813.0  1.333198e+09  1844.0  

# **Filtering Data**
Filtering the data so that only the necessary components are included in the dataset ('title', 'subreddit', 'score', 'username', 'total_votes','number_of_upvotes', 'number_of_downvotes', 'number_of_comments')

Explain why?
number of upvotes and downvotes rather than score of post to compare which one is higher.
- score just to see overall


In [ ]:
filtered_data = dataset[['title', 'subreddit', 'username', 'total_votes','number_of_upvotes', 'number_of_downvotes', 'number_of_comments']]
print(filtered_data)
print(filtered_data.head())
print(filtered_data['title'].compute())

Dask DataFrame Structure:
                title subreddit username total_votes number_of_upvotes number_of_downvotes number_of_comments
npartitions=1                                                                                                
               string    string   string     float64           float64             float64            float64
                  ...       ...      ...         ...               ...                 ...                ...
Dask Name: getitem, 3 expressions
Expr=ArrowStringConversion(frame=FromMapProjectable(34ca8c0))[['title', 'subreddit', 'username', 'total_votes', 'number_of_upvotes', 'number_of_downvotes', 'number_of_comments']]


/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


                             title subreddit             username  \
0           And here's a downvote.     funny  Animates_Everything   
1                      Expectation  GifSound        Gangsta_Raper   
2                         Downvote  GifSound        Gangsta_Raper   
3  Every time I downvote something  GifSound        Gangsta_Raper   
4   Downvote &quot;Dies Irae&quot;  GifSound        Gangsta_Raper   

   total_votes  number_of_upvotes  number_of_downvotes  number_of_comments  
0      63470.0            32657.0              30813.0               622.0  
1         35.0               29.0                  6.0                 3.0  
2         41.0               32.0                  9.0                 0.0  
3         10.0                6.0                  4.0                 0.0  
4         65.0               57.0                  8.0                 0.0  


/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


0                                    And here's a downvote.
1                                               Expectation
2                                                  Downvote
3                           Every time I downvote something
4                            Downvote &quot;Dies Irae&quot;
                                ...                        
132298                                           OM NOM NOM
132299                            Don't feed the animals...
132300                                          WTF worthy.
132301    Just a camel eating a kids head, welcome to th...
132302      Ass is looking at a little girl eaten by camel.
Name: title, Length: 132303, dtype: string


# **Cleaning Data**

We turn the titles into lower case to avoid the problem of duplicates with capitalization.

Afterwards, we remove stopwords to remove common words that don't add much meaning to our analysis as well as punctuation.


In [ ]:
import re
import pandas as pd
#turning titles into lowercase words to make it easier when analysing
#filtered_data['title'] = filtered_data['title'].str.lower()
#testing
#filtered_data['title'].compute()

#removing stopwords
#delete later: (list of words, subreddit, engagement score)
def clean_titles(title):
  valid_words = []
  if isinstance(title, str) and pd.notna(title):
    title = re.sub(r'[^\w\s]', '', title)
    for word in title.split(" "):
      if len(word) > 0 and word not in stop_words1:
        valid_words.append(word.lower())
  else:
    return[]
  return " ".join(valid_words)





Code to filter the titles

Engagement ratios are calculated to use as metrics

In [ ]:
filtered_data['title'] = filtered_data['title'].map(clean_titles, meta=filtered_data['title'])
#testing whether the titles are as expected
filtered_data_result = filtered_data['title'].compute()
print(filtered_data_result)
#printing the first 5 results

#Creating an engagement ratio to show how users interact with each post
filtered_data['engagement ratio'] = filtered_data['number_of_upvotes']/filtered_data['total_votes']
print(filtered_data.head())

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


0                                   and heres downvote
1                                          expectation
2                                             downvote
3                      every time i downvote something
4                           downvote quotdies iraequot
                              ...                     
132298                                      om nom nom
132299                               dont feed animals
132300                                      wtf worthy
132301    just camel eating kids head welcome internet
132302             ass looking little girl eaten camel
Name: title, Length: 132303, dtype: object


/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


                             title subreddit             username  \
0               and heres downvote     funny  Animates_Everything   
1                      expectation  GifSound        Gangsta_Raper   
2                         downvote  GifSound        Gangsta_Raper   
3  every time i downvote something  GifSound        Gangsta_Raper   
4       downvote quotdies iraequot  GifSound        Gangsta_Raper   

   total_votes  number_of_upvotes  number_of_downvotes  number_of_comments  \
0      63470.0            32657.0              30813.0               622.0   
1         35.0               29.0                  6.0                 3.0   
2         41.0               32.0                  9.0                 0.0   
3         10.0                6.0                  4.0                 0.0   
4         65.0               57.0                  8.0                 0.0   

   engagement ratio  
0          0.514527  
1          0.828571  
2          0.780488  
3          0.600000  
4     

# **Grouping words by their subreddits**

Grouping the words from post titles by their subreddits allow us to find words of importance in each subreddit (TF-IDF).
With this we may find common themes, and may be able to compare similarities between subreddits.

**To help perform TF-IDF, we will have to join all of the titles together to form a sort of document.**

In [ ]:
#creating a string combining all of the post titles SPECIFICALLY for TF-IDF
#group_data = filtered_data.groupby('subreddit')['title'].apply(lambda x: ' '.join(x)).reset_index()
#print(filtered_data['subreddit'].unique().compute())

group_data = filtered_data[['title', 'subreddit']].compute()
print(group_data.head())

subreddit_data = group_data.groupby('subreddit')['title'].apply(lambda x: ' '.join(x)).reset_index()
print(subreddit_data.head())
print(f"There are {subreddit_data['subreddit'].unique()} amount of rows")

for x in range(5):
  print(subreddit_data['title'][x])
  #print(subreddit_data['subreddit'][x])
#print(subreddit_data.compute())

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


                             title subreddit
0               and heres downvote     funny
1                      expectation  GifSound
2                         downvote  GifSound
3  every time i downvote something  GifSound
4       downvote quotdies iraequot  GifSound
   subreddit                                              title
0  2006Scape  as soon i finished manually installing cache h...
1     2XLite  my boyfriend brought home crapload chocolate w...
2     30ROCK                                im actually jealous
3        311                                    pumpkin carving
4      4chan  4chan really slow awesome slowpoke and thus go...
There are <ArrowStringArray>
[           '2006Scape',               '2XLite',               '30ROCK',
                  '311',                '4chan',             '911truth',
                 '9gag',              ': funny',                  'AMA',
 'A_Monocle_For_Sauron',
 ...
        'worldpolitics',                  'wow',          'wrongnumb

#**Grouping subreddits from metrics**

Creating a new dataframe that calculates metrics of the dataset. This will be merged with the dataset grouped by metrics. This way we have a string of titles that can be used as a document for TF-IDF, and engagement metrics that can be used to determine subreddits with popular posts and how they're interacted.



In [ ]:
import numpy as np
metric_data = filtered_data[['subreddit', 'total_votes', 'engagement ratio', 'number_of_comments']]
#print(metric_data.head())
#caclulating metrics
engagement_metadata = metric_data.groupby('subreddit').agg({'total_votes': np.mean, 'engagement ratio': np.mean, 'number_of_comments': np.mean}).compute().reset_index()
engagement_metadata.columns = ['subreddit', 'avg_total_votes', 'avg_engagement_ratio', 'avg_comments']


#adding a post count for each subreddit to ensure subreddits with less than x aren't included.
post_counts = filtered_data.groupby('subreddit').size().compute().reset_index(name='post_count')

#merging the post counts to match their subreddits
engagement_metadata = engagement_metadata.merge(post_counts, on='subreddit', how='left')

#keeping only subreddits where the post count > 100 to eliminate the possibility of low post count
#in a subreddit reddit but high number of votes that can skew the data.
engagement_metadata = engagement_metadata.query('post_count > 1000')

print(engagement_metadata.head())

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)
/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


  subreddit  avg_total_votes  avg_engagement_ratio  avg_comments  post_count
0     funny      2460.074172              0.596047     36.929501       55277
1  GifSound        27.007311              0.658663      0.873217        5608
2      gifs       884.646116              0.647087     17.233929       12538
4      pics      1866.567255              0.630789     48.873624       24712
5   atheism      1530.366310              0.649735     63.202614        3366


##**Merging the two datasets**

We will merge the two datasets to create a singular dataset that can be used for TF-IDF and engagement interpretation.

We will also be finding out the most engaged with subreddit. With this we may also find out how the subreddit is being interacted with.

In [ ]:
dataset = df.merge(engagement_metadata, subreddit_data, on='subreddit', how='inner')
#print(dataset.head())

ranked_dataset = dataset.sort_values(by='avg_total_votes', ascending=False).head(10)
print(ranked_dataset)



  subreddit  avg_total_votes  avg_engagement_ratio  avg_comments  post_count  \
0     funny      2460.074172              0.596047     36.929501       55277   
5    gaming      2065.452310              0.568028     64.352609        3009   
3      pics      1866.567255              0.630789     48.873624       24712   
6       WTF      1552.849012              0.579460     57.880587       12193   
4   atheism      1530.366310              0.649735     63.202614        3366   
7       aww      1488.294764              0.741335     19.505236        5920   
2      gifs       884.646116              0.647087     17.233929       12538   
1  GifSound        27.007311              0.658663      0.873217        5608   

                                               title  
0  and heres downvote getting first downvote new ...  
5  sitting new knight rgaming seeing insanely twi...  
3  every lastairbender post nsfw tag ohhh thats l...  
6  the faces definitely one weirder gifs ive come...  
4  t

# **TF-IDF implementation**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Creating a TF-IDF dataframe for manipulation
tfidf = TfidfVectorizer(stop_words = 'english')

tfidf_matrix = tfidf.fit_transform(ranked_dataset['title'])

tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    index=ranked_dataset['subreddit'],
    columns= tfidf.get_feature_names_out()
)


#Looking for the top 20 words
def top_20_words(row, n=10):
  top_words = row.argsort()[-n:][::-1]
  words_list = []
  for i in top_words:
    word = tfidf_df.columns[i]
    words_list.append((word, row.iloc[i]))
  return words_list

#looking for the top 20 words in each subreddit
tfidf_wordlist = tfidf_df.apply(lambda row: top_20_words(row), axis=1)
print(tfidf_wordlist)



subreddit
funny       [(feel, 0.41473858503511), (reddit, 0.26381010...
gaming      [(game, 0.48967949353646056), (steam, 0.238072...
pics        [(like, 0.2424917049179486), (picture, 0.22176...
WTF         [(wtf, 0.3680641077292314), (nsfw, 0.350736414...
atheism     [(jesus, 0.32237073296561986), (religion, 0.26...
aww         [(baby, 0.4528238432809415), (cute, 0.30622081...
gifs        [(feel, 0.3945808729910792), (gif, 0.336538612...
GifSound    [(gif, 0.2577075557981244), (cat, 0.2457608479...
dtype: object


# **Cosine Similarity**
Calculating cosine similarity will allow us to see which subreddits have high similarity. From here we may use this to compare how each subreddit is interacted with differently.

In [ ]:
def cosine_similarity(u, v):
    dot_product = np.dot(u,v)
    norm_u = (sum([(u[i])**2 for i in range(len(u))]))**(1/2)
    norm_v = (sum([(v[i])**2 for i in range(len(u))]))**(1/2)

    return dot_product/(norm_u * norm_v)


tfidf_wordlist_values = tfidf_wordlist.map(lambda row: [x[1] for x in row])

sub_names = tfidf_df.index.tolist()
vectors = tfidf_df.values
print(sub_names)






['funny', 'gaming', 'pics', 'WTF', 'atheism', 'aww', 'gifs', 'GifSound']


# **Comparing engagement ratios along with similarities**

The engagement ratio for each word is calculated by taking the average engagement ratio from all of the posts in the unique subreddit where the word is present.
This will allow use to see how differently posts with these specific words are interacted with.

In [ ]:
 #ranked_dataset.head(10)

#calculating the type of engagement each word has in their respective subreddits
tfidf_wordlist_values = tfidf_wordlist.map(lambda row: [x[0] for x in row])
print(tfidf_wordlist_values)

#using the filtered data to use the engagement ratio of each post
filtered_data['title_list'] = filtered_data['title'].str.split()
#print(filtered_data.head())

###Adding average engagement ratios for each individual word to show the type of engagement each word is associated with in posts in their respective subreddit
#Titles need to be exploded to help compute the engagement ratios
exploded_titles = filtered_data.explode('title_list')
word_engagement = (exploded_titles.groupby(['subreddit', 'title_list'])['engagement ratio'].agg(['mean', 'count']).reset_index()
    .rename(columns={'title_list': 'word'})
    .rename(columns={'mean': 'avg_engagement_ratio','count': 'post_count'})).compute()
#print(word_engagement)


#This list will be used to create a dataframe
rows = []
for subreddit, word_list in tfidf_wordlist.items():
    for word, score in word_list:
        rows.append({'subreddit': subreddit, 'word': word, 'tfidf_score': score})

#creates a dataframe that can be merged with word engagement (only the words that are in the top 20)w
top_words_flat = pd.DataFrame(rows)
merged = top_words_flat.merge(word_engagement, on=['subreddit', 'word'], how='left')
print(merged)


subreddit
funny       [feel, reddit, time, like, day, im, post, dont...
gaming      [game, steam, playing, time, gaming, games, fi...
pics        [like, picture, reddit, just, time, day, new, ...
WTF         [wtf, nsfw, dont, like, fuck, just, know, thin...
atheism     [jesus, religion, atheist, god, atheists, chri...
aww         [baby, cute, cat, little, just, kitty, dog, cu...
gifs        [feel, gif, time, reddit, day, like, post, im,...
GifSound    [gif, cat, like, fixed, im, song, dance, time,...
dtype: object


/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: ParserWarning: Skipping line 67828: expected 13 fields, saw 16
Skipping line 67829: expected 13 fields, saw 16
Skipping line 67830: expected 13 fields, saw 16
Skipping line 67831: expected 13 fields, saw 16
Skipping line 67832: expected 13 fields, saw 14

  df = reader(bio, **kwargs)


   subreddit    word  tfidf_score  avg_engagement_ratio  post_count
0      funny    feel     0.414739              0.610574        2965
1      funny  reddit     0.263810              0.601518        1886
2      funny    time     0.236114              0.605077        1688
3      funny    like     0.222546              0.600848        1591
4      funny     day     0.176946              0.604524        1265
..       ...     ...          ...                   ...         ...
75  GifSound    song     0.179323              0.640083          94
76  GifSound   dance     0.165547              0.688728          97
77  GifSound    time     0.151894              0.678329          89
78  GifSound    dont     0.134827              0.663854          79
79  GifSound   party     0.133120              0.613662          78

[80 rows x 5 columns]


# **Printing the results of TF-IDF and Cosine Similarity**

In [ ]:
grouped_by_subreddit = merged.groupby('subreddit')

#The type of engagement of each word
#for each subreddit, we print their rows.

print('TF-IDF Table: \n')
print('This calculates the number of a word to the subreddit.\n Average Word Engagement ratio takes the average engagement of all the posts the word is present in.\n')
print('Average Engagement Ratios < 0.5 mean posts containing the word have more negative interactions. Scores > 0.5 mean more positive interactions')
for subreddit_name, subreddit_data in grouped_by_subreddit:
    print(f"Subreddit: {subreddit_name}")

    subreddit_data_sorted = subreddit_data.sort_values(by='tfidf_score', ascending=False)

    for index, row in subreddit_data_sorted.iterrows():
        word = row['word']
        tfidf_score = row['tfidf_score']
        avg_engagement_ratio = row['avg_engagement_ratio']
        post_count = row['post_count'] # Include post count for more context
        print(f"  Word: {word} {' ' * (10- len(word))} TF-IDF Score: {tfidf_score:.4f}  {' ' * 5}Avg Word Engagement Ratio: {avg_engagement_ratio:.4f}")
    print("\n")

print('\nThe Cosine Similarity matrix shows the similarities between two subreddits.\nRatio scores closer to 1 indicate high similarity of emphasized words and topics')
#matrix for the cosine similarity
cos_sim_matrix = pd.DataFrame()
#values to be added into the matrix
for x in range(len(sub_names)):
    for y in range(len(sub_names)):
        similarity = cosine_similarity(vectors[x], vectors[y])
        cos_sim_matrix.loc[sub_names[x], sub_names[y]] = similarity
print(cos_sim_matrix)


TF-IDF Table: 

This calculates the number of a word to the subreddit.
 Average Word Engagement ratio takes the average engagement of all the posts the word is present in.

Average Engagement Ratios < 0.5 mean posts containing the word have more negative interactions. Scores > 0.5 mean more positive interactions
Subreddit: GifSound
  Word: gif         TF-IDF Score: 0.2577       Avg Word Engagement Ratio: 0.6379
  Word: cat         TF-IDF Score: 0.2458       Avg Word Engagement Ratio: 0.6738
  Word: like        TF-IDF Score: 0.2202       Avg Word Engagement Ratio: 0.6862
  Word: fixed       TF-IDF Score: 0.2065       Avg Word Engagement Ratio: 0.6514
  Word: im          TF-IDF Score: 0.1946       Avg Word Engagement Ratio: 0.6514
  Word: song        TF-IDF Score: 0.1793       Avg Word Engagement Ratio: 0.6401
  Word: dance       TF-IDF Score: 0.1655       Avg Word Engagement Ratio: 0.6887
  Word: time        TF-IDF Score: 0.1519       Avg Word Engagement Ratio: 0.6783
  Word: dont      